# **Squeeze-and-Excitation Networks**

Hu, J., Shen, L., & Sun, G. (2018). Squeeze-and-excitation networks. In Proceedings of the IEEE conference on computer vision and pattern recognition (pp. 7132-7141).

In [2]:
import tensorflow as tf
tf.__version__

'2.4.1'

In [31]:
def SE_Block(
    x,
    reduction_rate = 16,
):
    assert not (x.shape[-1] % reduction_rate), \
        f"x.shape {x.shape} must be divided by reduction_rate {r}"
    
    y = tf.keras.layers.GlobalAveragePooling2D()(x)[:, tf.newaxis, tf.newaxis, :]
    y = tf.keras.layers.Dense(y.shape[-1] // reduction_rate)(y)
    y = tf.keras.layers.Activation(tf.nn.relu)(y)
    y = tf.keras.layers.Dense(x.shape[-1])(y) ## recover the original channel
    y = tf.keras.layers.Activation(tf.nn.softmax)(y)
        
    x = tf.keras.layers.Multiply()([x, y]) ## channel-wise multiplication
    
    return x

    
def Conv2D_BN_ReLU(
    x, 
    filters, 
    kernel_size, 
    strides = 1,
    padding = "same",
    use_activation = True
):
    x = tf.keras.layers.Conv2D(filters, kernel_size, strides = strides, padding = padding)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    if use_activation:
        x = tf.keras.layers.Activation("relu")(x)

    return x


def residual_block(
    x, 
    output_filters, 
):
    assert x.shape[-1] <= output_filters, \
        "The size of the output dimension must be equal to or greater than the size of the input dimension."

    residual = x

    strides = 2 if (residual.shape[-1] < output_filters and output_filters !=  256) else 1

    x = Conv2D_BN_ReLU(x, output_filters // 4, 1, strides = strides)
    x = Conv2D_BN_ReLU(x, output_filters // 4, 3)
    x = Conv2D_BN_ReLU(x, output_filters, 1, use_activation = False)
    x = SE_Block(x)

    ## When the input and output are the same dimensions, the shortcut performs identity mapping.
    ## When the demensions increase, the projection shortcut in Eqn.(2) is used to match dimensions (done by 1x1 convolutions).
    if residual.shape[-1] < output_filters:
        residual = Conv2D_BN_ReLU(residual, output_filters, 1, strides = strides, use_activation = False)

    x = tf.keras.layers.Add()([x, residual])
    x = tf.keras.layers.Activation("relu")(x)

    return x

In [32]:
IMAGE_SIZE = [224, 224] # Cropped ImageNet test size.

def create_NN(
    model_name
):
    """Base ResNet50"""
    model_input = tf.keras.layers.Input(shape = (*IMAGE_SIZE, 1), dtype = tf.float32)

    ## Entry flow.
    x = Conv2D_BN_ReLU(model_input, 64, 7, strides = 2)
    x = tf.keras.layers.MaxPool2D(3, strides = 2, padding = "same")(x)

    ## Middle flow.
    for output_filters in [256] * 3 + [512] * 4 + [1024] * 6 + [2048] * 3:
        x = residual_block(x, output_filters)

    ## Exit flow.
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    model_output = tf.keras.layers.Dense(1000, activation = "softmax")(x)

    return tf.keras.Model(
        inputs = model_input,
        outputs = model_output,
        name = model_name)

In [ ]:
tmp = create_NN("tmp")
tmp.summary()

In [ ]:
tf.keras.utils.plot_model(tmp, show_shapes = True)

In [35]:
del tmp